In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d clmentbisaillon/fake-and-real-news-dataset

 98% 40.0M/41.0M [00:02<00:00, 23.5MB/s]
100% 41.0M/41.0M [00:02<00:00, 14.7MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/fake-and-real-news-dataset.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, LSTM, Activation
from tensorflow.keras.models import Sequential, load_model

In [ ]:
from nltk.tokenize import RegexpTokenizer

In [ ]:
real_news = pd.read_csv("/content/True.csv")

In [ ]:
real_news

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [ ]:
real_news['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [ ]:
text=list (real_news.text.values)

In [ ]:
join_text=" ".join(text)

In [ ]:
partial_text = join_text[:10000]

In [ ]:
tokenizer=RegexpTokenizer(r"\w+")

In [ ]:
token = tokenizer.tokenize(partial_text.lower())

In [ ]:
#token

In [ ]:
unique_token = np.unique(token)

In [ ]:
unique_token_index={tokens : idx for idx , tokens in enumerate(unique_token)}

In [ ]:
n_word=10
input_word=[]
next_word=[]

In [ ]:
for i in range(len(token)-n_word):
  input_word.append(token[i:i+n_word])
  next_word.append(token[i+n_word])

In [ ]:
x= np.zeros((len(input_word), n_word, len(unique_token)), dtype=bool)
y=np.zeros((len(next_word), len(unique_token)), dtype=bool)

In [ ]:
for i , words in enumerate(input_word):
  for j, word in enumerate(words):
    x[i, j, unique_token_index[word]]=1
  y[i, unique_token_index[next_word[i]]]=1

In [ ]:
model=Sequential()

In [ ]:
model.add(LSTM(128, input_shape=(n_word, len(unique_token)), return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_token)))
model.add(Activation("softmax"))

In [ ]:
from tensorflow.keras.optimizers import RMSprop

In [ ]:
model.compile(loss="categorical_crossentropy", optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])

In [ ]:
model.fit(x,y,batch_size=128,epochs=100, shuffle=True)

Epoch 1/100
13/13 [==============================] - 9s 9ms/step - loss: 6.1602 - accuracy: 0.0488
Epoch 2/100
13/13 [==============================] - 0s 8ms/step - loss: 5.8805 - accuracy: 0.0607
Epoch 3/100
13/13 [==============================] - 0s 8ms/step - loss: 5.8352 - accuracy: 0.0594
Epoch 4/100
13/13 [==============================] - 0s 7ms/step - loss: 5.8203 - accuracy: 0.0607
Epoch 5/100
13/13 [==============================] - 0s 8ms/step - loss: 5.8032 - accuracy: 0.0607
Epoch 6/100
13/13 [==============================] - 0s 7ms/step - loss: 5.7866 - accuracy: 0.0607
Epoch 7/100
13/13 [==============================] - 0s 7ms/step - loss: 5.7687 - accuracy: 0.0607
Epoch 8/100
13/13 [==============================] - 0s 7ms/step - loss: 5.7351 - accuracy: 0.0594
Epoch 9/100
13/13 [==============================] - 0s 7ms/step - loss: 5.6783 - accuracy: 0.0576
Epoch 10/100
13/13 [==============================] - 0s 7ms/step - loss: 5.6091 - accuracy: 0.0588
Epoch 11/

In [ ]:
model.save('textGen.h5')

In [ ]:
model=load_model("/content/textGen.h5")

In [ ]:
def predict_text_word(input_text, n_best):
  input_text= input_text.lower()
  x=np.zeros((1, n_word, len(unique_token)))
  for i, word in enumerate(input_text.split()):
    x[0,i,unique_token_index[word]] = 1
  prediction = model.predict(x)[0]

  return np.argpartition(prediction, n_best)[-n_best:]

In [ ]:
possible = predict_text_word("he will have to took in this think and he ", 10)

1/1 [==============================] - 1s 625ms/step


In [ ]:
possible

array([630, 631, 632,   4, 634, 635, 636, 637,   2,   0])

In [ ]:
print([unique_token[idx] for idx in possible])

['with', 'without', 'work', '2016', 'would', 'year', 'years', 'yet', '11', '1']


In [ ]:
import random

In [ ]:
from logging import exception
def generate_sentence(input_text, text_length, creativity=3):
  word_sequence=input_text.split()
  current=0
  for i in range(text_length):
    sub_sequence=" ".join(tokenizer.tokenize(" ".join(word_sequence)))[current:current+n_word]
    try:
      choice= unique_token[random.choice(predict_text_word(sub_sequence, creativity))]
    except:
      choice=random.choice(unique_token)
    word_sequence.append(choice)
    current+=1
  return " ".join(word_sequence)

In [ ]:
generate_sentence("how ", 2)

1/1 [==============================] - 0s 18ms/step


'how you indefinitely'